In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time
import datetime as dt
import importlib

import aux_funcs as aux

# Read test files

In [2]:
sample = aux.read_file('sample_submission.csv')

sample_submission.csv read in 5.4 s
Mem. usage decreased to 198.83 Mb (68.7% reduction)


In [2]:
weather_test_df = aux.read_file('weather_test.csv')

weather_test.csv read in 0.5 s
Mem. usage decreased to  6.08 Mb (68.1% reduction)


In [3]:
weather_test_df = aux.convert_timestamp(weather_test_df)

In [4]:
aux.null_data_stats(weather_test_df)

,total,percentage
cloud_coverage,140448,50.658808
precip_depth_1_hr,95588,34.478057
sea_level_pressure,21265,7.670167
wind_direction,12370,4.461790
wind_speed,460,0.165919
dew_temperature,327,0.117947
air_temperature,104,0.037512
timestamp,0,0.000000
site_id,0,0.000000


In [5]:
weather_test_df.loc['air_temperature'] = aux.nan_interpolate(weather_test_df['air_temperature'], verbose=1)

Series contains 104 nans distributed in 47 contiguous groups


C:\Users\Roberto\Anaconda3\lib\site-packages\pandas\core\indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
weather_test_df.columns

Index(['site_id', 'timestamp', 'air_temperature', 'cloud_coverage',
       'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
       'wind_direction', 'wind_speed'],
      dtype='object')

In [7]:
weather_test_df.drop(columns=['cloud_coverage','dew_temperature','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed'],inplace=True)
weather_test_df.head()

,site_id,timestamp,air_temperature
0,0.0,2017-01-01 00:00:00,17.796875
1,0.0,2017-01-01 01:00:00,17.796875
2,0.0,2017-01-01 02:00:00,16.093750
3,0.0,2017-01-01 03:00:00,17.203125
4,0.0,2017-01-01 04:00:00,16.703125


In [21]:
weather_test_df.site_id.max()

15.0

In [27]:
weather_test_df.timestamp.min()

Timestamp('2017-01-01 00:00:00')

In [23]:
weather_test_df.timestamp.max()

Timestamp('2018-12-31 23:00:00')

In [29]:
start = dt.datetime(2017,1,1)
print(start)
end = dt.datetime(2019,1,1,0,0,0)
print(end)

2017-01-01 00:00:00
2019-01-01 00:00:00


In [30]:
from datetime import datetime, timedelta

timestamps = np.arange(start, end, timedelta(hours=1)).astype(datetime)
len(timestamps)

17520

In [31]:
sites = np.arange(0,16)
sites

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [66]:
new_temperature = []

for site in [sites[0]]:
    subdf = weather_test_df.loc[weather_test_df['site_id']==site]
    new_temperature.append( np.interp(timestamps, subdf['timestamp'], subdf['air_temperature']))

TypeError: Cannot cast array data from dtype('O') to dtype('float64') according to the rule 'safe'

In [69]:
float(timestamps[0])

TypeError: float() argument must be a string or a number, not 'datetime.datetime'

In [58]:
ss= np.tile(sites, (1,17520))
ss= np.transpose(ss)
ss.shape

(280320, 1)

In [59]:
tts = np.tile(timestamps,(1,16))
tts = np.transpose(tts)
tts.shape

(280320, 1)

In [62]:
nn =np.concatenate((ss,tts),axis=1)
nn.shape

(280320, 2)

In [63]:
new_weather = pd.DataFrame(np.concatenate((ss,tts),axis=1) ,columns=['site_id','timestamp'])
new_weather.head()

,site_id,timestamp
0,0,2017-01-01 00:00:00
1,1,2017-01-01 01:00:00
2,2,2017-01-01 02:00:00
3,3,2017-01-01 03:00:00
4,4,2017-01-01 04:00:00


In [33]:
17520*16

280320

In [65]:
from scipy import interpolate
f = interpolate.interp2d(weather_test_df['site_id'], weather_test_df['timestamp'], weather_test_df['air_temperature'])

OverflowError: Too many data points to interpolate

In [34]:
weather_test_df.shape

(277244, 3)

In [8]:
# import test
test_df = aux.read_file('test.csv')

test.csv read in 35.4 s
Mem. usage decreased to 596.49 Mb (53.1% reduction)


In [9]:
# timestamp
test_df = aux.convert_timestamp(test_df)
test_df = aux.split_timestamp(test_df)

In [10]:
test_df.head()

,row_id,building_id,meter,timestamp,hour,day,year,weeknumber,weekday,month
0,0,0,0,2017-01-01,0,1,2017,52,6,1
1,1,1,0,2017-01-01,0,1,2017,52,6,1
2,2,2,0,2017-01-01,0,1,2017,52,6,1
3,3,3,0,2017-01-01,0,1,2017,52,6,1
4,4,4,0,2017-01-01,0,1,2017,52,6,1


In [60]:
test_df['timestamp'].nunique()

17520

In [61]:
weather_test_df['timestamp'].nunique()

17520

In [11]:
# import building
building_df = aux.read_file('building_metadata.csv')
#
building_df['log_square_feet'] = np.log(building_df['square_feet'])

building_metadata.csv read in 0.1 s
Mem. usage decreased to  0.03 Mb (60.3% reduction)


In [12]:
building_df.columns

Index(['site_id', 'building_id', 'primary_use', 'square_feet', 'year_built',
       'floor_count', 'log_square_feet'],
      dtype='object')

In [13]:
building_df.drop(columns=['year_built','floor_count'],inplace=True)
building_df.head()

,site_id,building_id,primary_use,square_feet,log_square_feet
0,0,0,Education,7432,8.913550
1,0,1,Education,2720,7.908387
2,0,2,Education,5376,8.589700
3,0,3,Education,23685,10.072597
4,0,4,Education,116607,11.666565


In [112]:
importlib.reload(aux)

<module 'aux_funcs' from 'C:\\rob\\Kaggle\\ASHRAE\\aux_funcs.py'>

In [14]:
#merge tables
testbw = test_df.join(building_df, on='building_id', rsuffix = 'r')
testbw.drop(columns=['building_idr'],inplace=True)
print(testbw.shape)
testbw.head()


(41697600, 14)


,row_id,building_id,meter,timestamp,hour,day,year,weeknumber,weekday,month,site_id,primary_use,square_feet,log_square_feet
0,0,0,0,2017-01-01,0,1,2017,52,6,1,0,Education,7432,8.913550
1,1,1,0,2017-01-01,0,1,2017,52,6,1,0,Education,2720,7.908387
2,2,2,0,2017-01-01,0,1,2017,52,6,1,0,Education,5376,8.589700
3,3,3,0,2017-01-01,0,1,2017,52,6,1,0,Education,23685,10.072597
4,4,4,0,2017-01-01,0,1,2017,52,6,1,0,Education,116607,11.666565


In [69]:
weather_test_df.groupby(['site_id'])['timestamp'].count().to_frame()

,timestamp
site_id,
0,17520
1,17287
2,17520
3,17517
4,17516
5,17241
6,17504
7,16573
8,17520


In [19]:
xx=test_df.groupby(['building_id','meter'])['timestamp'].count().to_frame()

In [20]:
xx['timestamp'].unique()

array([17520], dtype=int64)

In [70]:
max(test_df['timestamp'])

Timestamp('2018-12-31 23:00:00')

In [71]:
365*2*24

17520

In [17]:
testbw.drop(columns=['building_idr'],inplace=True)

In [15]:
#
testbw = pd.merge(testbw, weather_test_df,left_on=['site_id','timestamp'],right_on=['site_id','timestamp'],how='left')
print(testbw.shape)
testbw.head()

(41697600, 15)


,row_id,building_id,meter,timestamp,hour,day,year,weeknumber,weekday,month,site_id,primary_use,square_feet,log_square_feet,air_temperature
0,0,0,0,2017-01-01,0,1,2017,52,6,1,0,Education,7432,8.913550,17.796875
1,1,1,0,2017-01-01,0,1,2017,52,6,1,0,Education,2720,7.908387,17.796875
2,2,2,0,2017-01-01,0,1,2017,52,6,1,0,Education,5376,8.589700,17.796875
3,3,3,0,2017-01-01,0,1,2017,52,6,1,0,Education,23685,10.072597,17.796875
4,4,4,0,2017-01-01,0,1,2017,52,6,1,0,Education,116607,11.666565,17.796875


In [21]:
from sklearn.tree import DecisionTreeRegressor
# read trees
import pickle
with open('tree_of_trees_191023.pkl','rb') as file:
    dtrs = pickle.load(file)

In [22]:
dtrs[0]

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [23]:
testbw['prediction'] =0

In [24]:
subgroups = testbw.groupby(['primary_use','meter'])['meter'].count()
subgroups

primary_use                    meter
Education                      0        9351497
                               1        3756343
                               2        2359801
                               3        1216193
Entertainment/public assembly  0        3124457
                               1         768036
                               2         522181
                               3         279929
Food sales and service         0          87591
                               1          70072
                               2          35035
                               3          35036
Healthcare                     0         367580
                               1         227406
                               2         157653
                               3          52548
Lodging/residential            0        2527715
                               1         976368
                               2         783937
                               3         192712
Man

In [27]:
subgroups.index[0][0]

'Education'

In [28]:
features_list=['weeknumber','weekday','hour','log_square_feet','air_temperature']

In [41]:
subix = subgroups.index
for k in range(0, len(subgroups)):

    use = subix[k][0]
    meter = subix[k][1]
    
    print(k, use, meter)
    print('   Filter by use and meter...')
    # condition
    cond = (testbw['primary_use']== use) & (testbw['meter']==meter)
    # select features
    print('   Select features and datapoints...')
    X = testbw.loc[cond, features_list]
    #pick tree
    dtr = dtrs[k]
    #predict
    print('   Prediction...')
    testbw.loc[cond, 'prediction'] = dtr.predict(X)
    print('   Done!')
    print('---------------')
    print('')

0 Education 0
   Filter by use and meter...
   Select features and datapoints...
   Prediction...
   Done!
---------------

1 Education 1
   Filter by use and meter...
   Select features and datapoints...
   Prediction...
   Done!
---------------

2 Education 2
   Filter by use and meter...
   Select features and datapoints...
   Prediction...
   Done!
---------------

3 Education 3
   Filter by use and meter...
   Select features and datapoints...
   Prediction...
   Done!
---------------

4 Entertainment/public assembly 0
   Filter by use and meter...
   Select features and datapoints...
   Prediction...
   Done!
---------------

5 Entertainment/public assembly 1
   Filter by use and meter...
   Select features and datapoints...
   Prediction...
   Done!
---------------

6 Entertainment/public assembly 2
   Filter by use and meter...
   Select features and datapoints...
   Prediction...
   Done!
---------------

7 Entertainment/public assembly 3
   Filter by use and meter...
   Selec

In [37]:
sum(testbw['prediction']!=0)

3208983

In [43]:
testbw.loc[:,['row_id','prediction']].to_csv('submission_01_191023.csv')

In [45]:
sample = aux.read_file('sample_submission.csv')
sample.head()

sample_submission.csv read in 14.7 s
Mem. usage decreased to 198.83 Mb (68.7% reduction)


,row_id,meter_reading
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [48]:
txt = []
with open('submission_01_191023.csv','r') as subfile:
    
    for k in range(0,5):
        txt.append(subfile.readline())

In [49]:
txt

[',row_id,prediction\n',
 '0,0,5.548920631408691\n',
 '1,1,4.003429412841797\n',
 '2,2,1.7773901224136353\n',
 '3,3,5.286128520965576\n']

In [50]:
txtsamp = []
with open('sample_submission.csv','r') as subfile:
    
    for k in range(0,5):
        txtsamp.append(subfile.readline())
txtsamp

['row_id,meter_reading\n', '0,0\n', '1,0\n', '2,0\n', '3,0\n']

In [53]:
submission = testbw.loc[:,['row_id','prediction']]
submission['meter_reading'] = np.exp(submission['prediction']-1)
submission.head()

,row_id,prediction,meter_reading
0,0,5.548921,94.530320
1,1,4.003429,20.154537
2,2,1.777390,2.175786
3,3,5.286129,72.684526
4,4,7.028724,415.184996


In [54]:
submission.drop(columns=['prediction'],inplace=True)
submission.head()

,row_id,meter_reading
0,0,94.530320
1,1,20.154537
2,2,2.175786
3,3,72.684526
4,4,415.184996


In [55]:
aux.null_data_stats(submission)

,total,percentage
meter_reading,0,0.0
row_id,0,0.0


In [56]:
submission.to_csv('submission_01_191024.csv',index=False)

In [57]:
submission.shape

(41498571, 2)

In [58]:
test_df.shape

(41697600, 10)